# Multiprocessing in Python

What happens if we need to do a lot of computation, and vectorization with NumPy is not enough? 

Basically: How can we assign different tasks to different cores or processors?

## Multiprocessing systems/interfaces

There are a few types of multiprocessing interfaces we can use, in order of easy to difficult:

| Interface | When to use |
|-----------|-------------|
| concurrent.futures.ProcessPoolExecutor | The modern way to launch parallel tasks. Usable for everything that takes a long time and needs to run on only one computer. |
| mpi4py.futures.MPIPoolExecutor | The modern way to run scalable parallel tasks on computer clusters. Use for long tasks that need to run on more than one computer. |
| multiprocess | An improved version of the built-in `multiprocessing` library. A bit more manual than the pools. |
| multiprocessing | The classic built-in library.  You probably will be using a part of this (like the Manager), but other components are trickier than the concurrent.futures implementation. |
| mpi4py classic | A bit beyond the scope of this class. |


In [1]:
import concurrent.futures
import time
import long_functions

In [4]:
def serial_task():
    possible_params = [3, 5, 10, 15]
    starting_time = time.time()
    for n in possible_params:
        long_functions.really_hard_task(n)
    duration_time = time.time() - starting_time
    print(f"Took {duration_time:.2f} s")

In [5]:
serial_task()

finished task crunching 3 with result 0.46
finished task crunching 5 with result 0.22
finished task crunching 10 with result 0.59
finished task crunching 15 with result 0.12
Took 8.03 s


In [6]:
possible_params = [3, 5, 10, 15]
with concurrent.futures.ProcessPoolExecutor(max_workers=4) as ppe:
    starting_time = time.time()
    the_futures = ppe.map(long_functions.really_hard_task, possible_params)
    print(list(the_futures))
    duration_time = time.time() - starting_time
    print(f"Took {duration_time:.2f} s")

[0.286835805546536, 0.473248040752672, 0.3926256908278173, 0.28259504313642336]
Took 2.61 s


What happens if we run too many workers at once?

If you want to run a lot of workers, you can try it on DevCloud!

## Basic Linear Algebra Problems in Multiprocessing

Turns out that the Math Kernel Library or any other basic linear algebra system (BLAS) sometimes will try to be sneaky and do multiprocessing/multithreading on its own.  However, this conflicts with your own code, so make sure to do this before importing numpy or any other packages:

```
import os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

# now you can import numpy and other packages
import numpy as np
import scipy.stats
# etc.
```